In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib
import numpy as np

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['font.size'] = 20

chip_path   = "HOXB13_chipseq_chr17_59000kb_60000kb.txt"
cpg_summary = "HOXB13_CpG_methylation_summary_3cat_per_kb.csv"   

region_chr = "chr17"
xmin_kb, xmax_kb = 59700, 59900

colors = {
    'chip_seq': '#FE9A37',        
    'unmethylated': '#F8766D',    
    'intermediate': '#FFD700',   
    'methylated': '#00BFC4',     
}

chip_cols = ["chrom","start","end","name","score","strand","signalValue","pValue","qValue","peak"]
chip = pd.read_csv(chip_path, header=0, names=chip_cols)
chip = chip[chip["chrom"] == region_chr].copy()

bin_edges = np.arange(xmin_kb*1000, (xmax_kb+1)*1000, 1000)
chip["bin"] = pd.cut(chip["start"], bins=bin_edges, labels=bin_edges[:-1]//1000)
chip_binned = chip.groupby("bin", dropna=True)["score"].max().reset_index()
chip_binned["bin"] = chip_binned["bin"].astype(int)
chip_binned["score_norm"] = chip_binned["score"] / 1000 * 1000
chip_binned["width_kb"] = 1

df_cpg = pd.read_csv(cpg_summary)
df_cpg["bin"] = (df_cpg["start"] // 1000).astype(int)
df_cpg = df_cpg[(df_cpg["bin"] >= xmin_kb) & (df_cpg["bin"] <= xmax_kb)]

fig = plt.figure(figsize=(14, 4))  
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1])

ax1 = fig.add_subplot(gs[0, 0])
ax1.bar(chip_binned["bin"], chip_binned["score_norm"],
        width=1, align="edge", color=colors['chip_seq'],
        edgecolor="white", linewidth=0.4)
ax1.set_ylabel("ChIP-seq\nSignal")
ax1.set_xlim(xmin_kb, xmax_kb)
ax1.set_ylim(0, 1000)
ax1.tick_params(labelbottom=False, labelsize=16)

ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)
ax2.bar(df_cpg["bin"], df_cpg["unmethylated_ratio"], width=1, 
        color=colors['unmethylated'], edgecolor="white", linewidth=0.2, label="Unmethylated")
ax2.bar(df_cpg["bin"], df_cpg["intermediate_ratio"], width=1, 
        bottom=df_cpg["unmethylated_ratio"],
        color=colors['intermediate'], edgecolor="white", linewidth=0.2, label="Intermediate")
ax2.bar(df_cpg["bin"], df_cpg["methylated_ratio"], width=1, 
        bottom=df_cpg["unmethylated_ratio"] + df_cpg["intermediate_ratio"],
        color=colors['methylated'], edgecolor="white", linewidth=0.2, label="Methylated")
ax2.set_ylabel("Methylated\nCpG (%)")
ax2.set_xlim(xmin_kb, xmax_kb)
ax2.set_ylim(0, 1)
ax2.tick_params(labelsize=16)
ax2.legend(fontsize=12, bbox_to_anchor=(1.02, 1), loc='upper left', frameon=False)
ax2.set_xlabel("Genome Position (kb)")

# fig.tight_layout()
# plt.savefig('HOXB13_mIDEA_genomic_2track.pdf', format='pdf', bbox_inches='tight')
plt.show()